<a href="https://colab.research.google.com/github/atacankavdir/academic/blob/main/data_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flipside

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 893.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 req

In [ ]:
import pandas as pd
from flipside import Flipside

In [ ]:
# Initialize `Flipside` with your API Key and API Url
flipside = Flipside("30a41e48-e8b2-4aae-9b4a-0162750b3712", "https://api-v2.flipsidecrypto.xyz")

sql = """
SELECT *
  FROM
      (
      SELECT FROM_ADDRESS, TO_ADDRESS,
             date_trunc('hour', block_timestamp) as hour,
             ROUND(SUM(AMOUNT_PRECISE)) AS value_eth,
             ROUND(SUM(AMOUNT_USD)) AS value_usd,
             count(TX_HASH) as n_of_transfers
        FROM ETHEREUM.CORE.EZ_NATIVE_TRANSFERS
       WHERE block_timestamp >= '2022-01-01'
       GROUP BY FROM_ADDRESS, TO_ADDRESS, date_trunc('hour', block_timestamp)
      )
  WHERE value_eth > 100
"""

# Run the query against Flipside's query engine and await the results
query_result_set = flipside.query(sql)

In [ ]:
def pagination_function(query_result_set): # what page are we starting on?
  current_page_number = 1

  # How many records do we want to return in the page?
  page_size = 10000

  # set total pages to 1 higher than the `current_page_number` until
  # we receive the total pages from `get_query_results` given the
  # provided `page_size` (total_pages is dynamically determined by the API
  # based on the `page_size` you provide)
  total_pages = 2

  # we'll store all the page results in `all_rows`
  all_rows = []

  while current_page_number <= total_pages:
    results = flipside.get_query_results(
      query_result_set.query_id,
      page_number=current_page_number,
      page_size=page_size
    )

    total_pages = results.page.totalPages
    if results.records:
        all_rows = all_rows + results.records

    current_page_number += 1
  return all_rows

In [ ]:
all_rows = pagination_function(query_result_set)

In [ ]:
eth_stats_hourly = pd.DataFrame(all_rows)

In [ ]:
eth_stats_hourly.hour.min()

'2022-01-01T00:00:00.000Z'

In [ ]:
import os
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
eth_stats_hourly.to_parquet('/content/drive/My Drive/DS Masters/Data/flipside_snowflake_eth_trans/eth_hourly_100_eth.parquet')